# WebScrapping CMF
Para extraer en una tabla Excel El Balance y Estado de Resultados de cualquier empresa.

In [1]:
#Como siempre importamos las librerias necesarias para ejecutar nuestro codigo.
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time

En el siguiente codigo se necesitan 2 variables que debemos rescatar manualmente y son:
- Primero buscar la empresa que queremos mirar y entrar a la pestaña fiscalizados, en este ejemplo utilizaremos a la empresa Falabella *https://www.cmfchile.cl/portal/principal/613/w3-search.php?keywords=falabella#fiscalizados* y guardaremos el link dentro de la lista __Empresa__ como un string en la posicion $[0]$. 
- En segundo lugar el rut que queremos ver, en este caso __90749000__ ojo, tambien como string, lo guardaremos en la posicion $[1]$.


*Debe utilizar Google Chrome*\
*La tercera variable de la lista Empresa debemos dejarla tal como está*

<img src="https://raw.githubusercontent.com/JoseZamoraCifras/WebScrapping-CMF/main/Screenshot_2.png">

In [2]:
Empresa = ['https://www.cmfchile.cl/portal/principal/613/w3-search.php?keywords=falabella#fiscalizados',"//td[text()='90749000']","./following-sibling::td/a"]
keys = ["Url", "selector1", "Selector2"]

Respecto al periodo que queremos extraer necesitaremos manipularlo en la lista __configurador__, para ello, debemos seleccionar 3 variables:
1. Año del cual extraeremos la información.
2. Quarter que queremos extraer.
3. Tipo de Norma, si es Estandar IFRS o Norma Chilena, en mi experiencia o al menos en las empresas que yo he mirado, generalemnte se utiliza el Estandar IFRS, ahí deben ir probando.

<img src="https://raw.githubusercontent.com/JoseZamoraCifras/WebScrapping-CMF/main/Screenshot_3.png">

Solo como orientación dejo el orden de los periodos:
- año ["Año" ,2023, 2022, 2021, 2020, 2019, 2018...etc]
- Quarter [Q4, Q3, Q2, Q1]
- Tipo Norma ["Seleccione", "Estandar IFRS", "Norma Chilena"]

En el caso que queramos extraer el Q3 del 2021, tipo norma IFRS deberiamos usar la siguiente configuración:
- configurador [4,1,1]

In [3]:
año = ["Año" ,2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 20211, 2010, 2009, 2008, 2007, 2006, 2005]
quarter = ["Q4", "Q3", "Q2", "Q1"]
Tipo_Norma = ["Seleccione", "Estandar IFRS", "Norma Chilena"]


#----------Reemplazar valores en la siguiente lista ----------------------------
configurador = [3,1,1]

Ya nos vamos manos a la obra y empezar a manipular con Selenium

In [4]:
# Inicializar el navegador web
driver = webdriver.Chrome()

# Cargar la página de inicio de la empresa en este ejemplo, 
driver.get(Empresa[0])
time.sleep(2)
td_element = driver.find_element(By.XPATH,Empresa[1])
a_element = td_element.find_element(By.XPATH,Empresa[2])
a_element.click()
time.sleep(2)


estados_financieros = driver.find_element(By.XPATH, "//*[@id='listado_reportes']/li[3]/a")
estados_financieros.click()
time.sleep(2)

from selenium.webdriver.support.ui import Select

periodo = driver.find_element(By.ID,'mm')
# Crear un objeto Select a partir del elemento <select>
select_periodo = Select(periodo)
# Obtener todas las opciones del <select> en una lista
base_periodo = select_periodo.options
#--------------------------------------------------------
año = driver.find_element(By.ID,'aa')
# Crear un objeto Select a partir del elemento <select>
select_año = Select(año)
# Obtener todas las opciones del <select> en una lista
base_año = select_año.options
#--------------------------------------------------------
tipo_norma = driver.find_element(By.NAME,'tipo_norma')
# Crear un objeto Select a partir del elemento <select>
select_tipo_norma = Select(tipo_norma)
# Obtener todas las opciones del <select> en una lista
base_tipo_norma = select_tipo_norma.options

base_año[configurador[0]].click()
base_periodo[configurador[1]].click()
base_tipo_norma[configurador[2]].click()

time.sleep(2)
consulta = driver.find_element(By.XPATH, '//input[@alt="Consultar"]')
consulta.click()

Bien, ya accedimos a la información, puedes ver en la pestaña emergente que se abrió con Selenium la información del Balance y Estados de resultados de Falabella para el periodo seleccionado.\
Ahora la parte practica es que queremos pasar la información a una tabla Excel para poder manipular la información en esa herramienta.

## Primero realizamos la tabla del Balance

In [5]:
#Ruta donde guardaremos nuestros Excel
ruta = "P:/Control de gestión/Reportes/2022/Flash 2.0/CMF"

In [6]:
tabla_html = """
#<table>
#    ...
#</table>
"""

soup = BeautifulSoup(tabla_html, 'html.parser')
tabla = soup.find('table')
filas = tabla.find_all('tr')
        
tabla = driver.find_element(By.TAG_NAME,'table')

encabezados = []
for encabezado in tabla.find_elements(By.TAG_NAME,'th'):
    encabezados.append(encabezado.text)
datos = []
for fila in tabla.find_elements(By.TAG_NAME,'tr'):
    fila_datos = []
    for celda in fila.find_elements(By.TAG_NAME,'td'):
        fila_datos.append(celda.text)
    if fila_datos:
        datos.append(fila_datos)
df = pd.DataFrame(datos, columns=encabezados)

In [9]:
df

,.,2021-06-30,2020-12-31,2020-06-30
0,Activos [sinopsis],None,None,None
1,Negocios no bancarios [sinopsis],None,None,None
2,Activos corrientes [sinopsis],None,None,None
3,Efectivo y equivalentes al efectivo,814.874.990,1.216.193.920,-
4,Otros activos financieros corrientes,40.912.734,41.163.435,-
...,...,...,...,...
100,Otras reservas,-475.110.591,-407.289.735,-
101,Patrimonio atribuible a los propietarios de la...,5.238.476.641,5.021.739.662,-
102,Participaciones no controladoras,955.664.547,963.516.029,-
103,Patrimonio total,6.194.141.188,5.985.255.691,6.141.290.895


##  Tabla del Estado de Resultados

In [7]:
tabla_eerr = driver.find_element(By.XPATH,"//*[@id='ERF']/table")
filas_eerr =driver.find_elements(By.TAG_NAME, "tr")

encabezados = []
for encabezado in tabla_eerr.find_elements(By.TAG_NAME,'th'):
    encabezados.append(encabezado.text)
datos_eerr = []
for fila in tabla_eerr.find_elements(By.TAG_NAME,'tr'):
    fila_datos_eerr = []
    for celda in fila.find_elements(By.TAG_NAME,'td'):
        fila_datos_eerr.append(celda.text)
    if fila_datos_eerr:
        datos_eerr.append(fila_datos_eerr)
df_eerr = pd.DataFrame(datos_eerr, columns=encabezados)

#Con esto cerramos el dataframe
driver.close()

In [8]:
df_eerr

,.,Desde 2021-01-01 Hasta 2021-06-30,Desde 2021-04-01 Hasta 2021-06-30,Desde 2020-01-01 Hasta 2020-06-30,Desde 2020-04-01 Hasta 2020-06-30
0,Ganancia (pérdida) [sinopsis],None,None,None,None
1,Negocios no bancarios [sinopsis],None,None,None,None
2,Ingresos de actividades ordinarias,4.608.667.199,2.390.588.744,3.346.062.707,1.458.364.095
3,Costo de ventas,3.187.574.085,-,2.364.939.732,-
4,Ganancia bruta,1.421.093.114,-,981.122.975,-
...,...,...,...,...,...
63,Ganancia (pérdida) por acción básica,"0,11",-,"-0,04",-
64,Ganancias por acción diluidas [sinopsis],None,None,None,None
65,Ganancias (pérdida) diluida por acción procede...,"0,11",-,"-0,04",-
66,Ganancias (pérdida) diluida por acción procede...,-,-,-,-


Ya generamos los DataFrame y ahora generamos nuestros Excel.

In [ ]:
Periodo = quarter[configurador[1]]
año_periodo = año[configurador[0]]
nombre_EF = "Estados Financieros " + año_periodo +" " + Periodo + ".xlsx"
nombre_ER = "Estados de Resultados " + año_periodo +" "+ Periodo + ".xlsx"

df.to_excel(ruta + "/" + nombre_EF)
df_eerr.to_excel(ruta + "/" + nombre_ER)